In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")'

In [ ]:
#groq = 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

In [37]:
%pip install langchain-groq

%pip install -qU langchain-groq




[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [39]:
%pip install -U langgraph


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
%pip install spotipy
%pip install dataclasses
%pip install typing


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26365 sha256=b43bacaf14d6c13df5e8b1346b9a0a75b276a22f04ab9cee8859471a347ba0f8
  Stored in directory: /Users/shubhamdwivedi/Library/Caches/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note:

## INITIALIZED THE STATE 
Also for saving the state we're using a static method and storing it in a json file in the project dir


In [ ]:
import json
from langgraph.graph import StateGraph, END
from langgraph.graph import StateGraph, END
from dataclasses import dataclass, field
from typing import List, Dict, Any

# Define the state (with a method for saving)
''' OLD METHOD
class SpotifyState(dict):
    playlists: list = []
    selected_playlist_name: str = None
    selected_playlist_id: str = None
    tracks: list = []
    user_input: str = None
    current_node: str = None
    messages: list = [] # Add messages list to the state
    tool_result: dict = None # Add tool_result to the state
'''

@dataclass
class SpotifyState:
    playlists: List[str] = field(default_factory=list)
    selected_playlist_name: str = None
    selected_playlist_id: str = None
    tracks: List[str] = field(default_factory=list)
    user_input: str = None
    current_node: str = "get_playlists"  # Initialize the starting node
    messages: List[Dict[str, str]] = field(default_factory=list)  # Store messages as dicts
    tool_result: Dict[str, Any] = None

    def to_dict(self): # Convert to dict for LangGraph
        return self.__dict__

    @staticmethod
    def from_dict(data): # Create from dict
        return SpotifyState(**data)
    def save(self, filename="spotify_state.json"):
        with open(filename, "w") as f:
            # Convert messages to simple dictionaries for JSON serialization
            serializable_state = self.copy() # create a copy to avoid modifying the original
            serializable_state["messages"] = [m.to_dict() for m in self["messages"]]  # Convert messages
            json.dump(serializable_state, f)

    @staticmethod
    def load(filename="spotify_state.json"):
        try:
            with open(filename, "r") as f:
                loaded_state = json.load(f)
                
                state = SpotifyState()
                # Convert loaded messages back to Message objects
                loaded_state["messages"] = [
                    HumanMessage(**m) if m["type"] == "human" else SystemMessage(**m) if m["type"] == "system" else m for m in loaded_state.get("messages", [])
                ]
                state.update(loaded_state)
                return state
        except FileNotFoundError:
            return SpotifyState()

# PromptTemplates + Spotify Oauth + Node functions

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Initialize Spotify API
SCOPE = "playlist-modify-public playlist-read-private playlist-modify-private"  # Add other scopes as needed
sp_oauth = SpotifyOAuth(client_id="4a8fdc631a704303ab465a07d5141a61",  # Replace with your credentials
                            client_secret="d649ba5407504a928eb2b94b49fa234d",
                            redirect_uri="http://localhost:8888/callback",  # Configure a redirect URI
                            scope=SCOPE)

sp = spotipy.Spotify(auth_manager=sp_oauth)

# Initialize LLM
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Defining prompts
playlist_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that lists user's spotify playlist and ask which one to modify."),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])

track_listing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that lists the tracks in a playlist and asks the user what to do next."),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])

add_track_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that confirms adding a track to a playlist."),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])

''' ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that lists the tracks in a playlist and ask the user what to do next."),
    ("human", "Sure, here are all the tracks in {playlist_name}:\n{tracks}\nWhat would you like to do? (add, delete, or say 'done')")
])'''
delete_track_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that confirms deletion of a track from the playlist, and ask the user what to do next"),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])

create_playlist_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that confirms creation of a new playlist. Ask the user for the playlist name"),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])

delete_playlist_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that confirms deletion of playlist and ask user what to do next? or (add, delete, or say 'done') "),
    MessagesPlaceholder("human_message")  # Placeholder for the human message
])



# Define nodes
def get_playlists(state):
    results = sp.current_user_playlists()
    playlists = [f"{item['name']} (ID: {item['id']})" for item in results['items']]
    state.playlists = playlists
    return {"playlists": playlists}  # Return an empty dict and the next node

def select_playlist(state, playlists):  # playlists is passed as argument
    playlist_name = state.get("user_input")
    playlist_id = None
    for playlist_str in playlists:
        if playlist_name in playlist_str:
            playlist_id = playlist_str.split("ID: ")[1].replace(")","")
            break
    if playlist_id:
        results = sp.playlist_items(playlist_id)
        tracks = [item['track']['name'] for item in results['items']]
        state["tracks"] = tracks
        state["selected_playlist_name"] = playlist_name
        state["selected_playlist_id"] = playlist_id
        return {}, "list_tracks" # Returning empty dict is correct
    else:
        return {}, "playlist_not_found" 

def list_tracks(state, selected_playlist_name, tracks): # Add selected_playlist_name and tracks as arguments
    return {}, "assistant"

'''
#def OLDER_METHOD_add_track(state):
    song_info = state.get("user_input").split(" by ")  # Assumes format: "Song Name by Artist Name"
    if len(song_info) == 2:
        song_name, artist_name = song_info
        # Search for the track (you'll need to handle potential errors/multiple results)
        results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track')
        if results['tracks']['items']:
            track_uri = results['tracks']['items'][0]['uri']
            sp.playlist_add_items(state["selected_playlist_id"], [track_uri])
            return {}, "list_tracks"  # Go back to listing tracks
        else:
            return {}, "track_not_found"
    else:
        return {}, "invalid_song_format"
'''

def add_track(state, tool_result):  # Correct: tool_result from state
    track_uri = tool_result['track_uri']
    track_name = tool_result['track_name']
    artist_name = tool_result['artist_name']
    sp.playlist_add_items(state["selected_playlist_id"], [track_uri])
    state["added_track"] = track_name # Update the state
    state["added_artist"] = artist_name # Update the state
    return {}, "list_tracks"


def delete_track(state):
    track_name = state.get("user_input")
    tracks_in_playlist = sp.playlist_items(state["selected_playlist_id"])
    track_id_to_remove = None
    for item in tracks_in_playlist['items']:
        if item['track']['name'] == track_name:
            track_id_to_remove = item['track']['id']
            break
    if track_id_to_remove:
        sp.playlist_remove_all_occurrences_of_items(state["selected_playlist_id"], [track_id_to_remove])
        return {}, "list_tracks"
    else:
        return {}, "track_not_found"

def create_playlist(state):
    playlist_name = state.get("user_input")
    sp.user_playlist_create(sp.me()['id'], playlist_name)
    return {}, "get_playlists" # Refresh playlists

def delete_playlist(state):
    playlist_name = state.get("user_input")
    playlist_id_to_delete = None
    for playlist_str in state["playlists"]:
      if playlist_name in playlist_str:
        playlist_id_to_delete = playlist_str.split("ID: ")[1].replace(")","")
        break
    if playlist_id_to_delete:
        sp.current_user_unfollow_playlist(playlist_id_to_delete)
        return {}, "get_playlists"
    else:
        return {}, "playlist_not_found"

def playlist_not_found(state):
  return {}, "get_playlists"

def track_not_found(state):
  return {}, "list_tracks"  # Or handle differently

def invalid_song_format(state):
    return {}, "add_track"

def invalid_action(state):
    return {}, "list_tracks"  # Or handle differently



### SEARCH_TRACKS tool def
+  Assistant Nodes which have implicit routing
# FIRST WITHOUT JSON PARSING IN TEMPLATES

In [55]:
from langchain.tools import Tool
from langchain_groq import ChatGroq

# Define the search track tool
def search_track_tool(song_name: str, artist_name: str): 
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track')
    if results['tracks']['items']:
        track_uri = results['tracks']['items'][0]['uri']
        return {"track_uri": track_uri, "track_name": results['tracks']['items'][0]['name'], "artist_name": results['tracks']['items'][0]['artists'][0]['name']}  # Return a dictionary with the track URI and other info
    else:
        return None  # Or raise an exception, or return a specific error value


from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
tools = [
    Tool(
        name="SearchTrack",
        func=search_track_tool,
        description="Searches for a track on Spotify given the song name and artist name.  Returns a dictionary containing the track URI, track name and artist name. Returns None if the track is not found.",
    )
]

llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(content="You are a helpful assistant that helps user in using Spotify and enhancing their User Experience.")

# ASSISTANT Nodes
def assistant(state: SpotifyState):
    messages = state.get("messages", [])
    user_input = state.get("user_input")

    if state.get("current_node") == "get_playlists":
        prompt = playlist_selection_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
        messages.append(prompt) # add the formatted prompt to the messages

    elif state.get("current_node") == "list_tracks":
        prompt = track_listing_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
        messages.append(prompt) # add the formatted prompt to the messages

    elif state.get("current_node") == "add_track":
        prompt = add_track_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
        messages.append(prompt) # add the formatted prompt to the messages


    elif state.get("current_node") == "delete_track":
            prompt = delete_track_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
            messages.append(prompt) # add the formatted prompt to the messages
    
    elif state.get("current_node") == "create_playlist":
            prompt = create_playlist_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
            messages.append(prompt) # add the formatted prompt to the messages

    elif state.get("current_node") == "delete_playlist":
            prompt = delete_playlist_prompt.format(human_message=HumanMessage(content=user_input))  # Format with user input
            messages.append(prompt)
    # ... (Add other prompts for delete_track, create_playlist, delete_playlist)
    else:
      messages.append(HumanMessage(content=user_input)) # default case

    response = llm_with_tools.invoke(messages) # invoke the LLM with the messages
    messages.append(response) # add the response to the messages
    return {"messages": messages}, "process_assistant_response"


    
def process_assistant_response(state: SpotifyState): 
    assistant_response = state["messages"][-1]  # Get the last message (assistant's response)

    if "SearchTrack" in assistant_response.content:  # Check if the tool was invoked
        try:
            # Improved extraction of tool arguments (more robust)
            tool_call_part = assistant_response.content.split("Tool: SearchTrack")[-1].strip()
            if "Args:" in tool_call_part: # Check if Args are present
              args_part = tool_call_part.split("Args: ")[1]
              # Handle cases with or without quotes around arguments
              if '"' in args_part:
                  parts = [p.strip().replace('"', '') for p in args_part.split(",")]
              else:
                  parts = [p.strip() for p in args_part.split(",")]
              song_name = parts[0]
              artist_name = parts[1]

              tool_result = search_track_tool(song_name, artist_name)
              if tool_result:
                  return {"tool_result": tool_result}, "add_track"
              else:
                  return {}, "track_not_found"
            else:
              return {}, "invalid_song_format" # Handle cases where tool args are missing

        except Exception as e:
            print(f"Error extracting tool args: {e}")
            return {}, "invalid_song_format"
    else:
        return {}, "invalid_action"  # Handle cases where tool was not invoked# Handle cases where tool was not invoked



##  BELOW CELLS NOT TESTED YET, ADDS JSON PARSING IN THE OUTPUT OF ASSISTANT NODE FOR PROCESS_ASSISTANT TO PERFORM CHECKS EFFICIENTLY 

In [ ]:
import json
from langgraph.graph import StateGraph, END
# ... (other imports)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessage, SystemMessage

# ... (Spotify API initialization, tool definition remain the same)

# ... (LLM initialization and tool binding remain the same)

# Define prompts (updated to instruct JSON format)
playlist_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that lists user's spotify playlist and ask which one to modify. Respond with JSON. If you need a tool, use this format: ```json\n{{\"action\": \"tool_call\", \"tool\": \"ToolName\", \"arguments\": [\"arg1\", \"arg2\"]}}\n```. If no tool is needed, respond with: ```json\n{{\"action\": \"no_tool\", \"response\": \"Your response here\"}}\n```"),
    MessagesPlaceholder("human_message")
])

track_listing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that lists the tracks in a playlist and asks the user what to do next. Respond with JSON. If you need a tool, use this format: ```json\n{{\"action\": \"tool_call\", \"tool\": \"ToolName\", \"arguments\": [\"arg1\", \"arg2\"]}}\n```. If no tool is needed, respond with: ```json\n{{\"action\": \"no_tool\", \"response\": \"Your response here\"}}\n```"),
    MessagesPlaceholder("human_message")
])

add_track_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that confirms adding a track to a playlist. Respond with JSON. If you need a tool, use this format: ```json\n{{\"action\": \"tool_call\", \"tool\": \"ToolName\", \"arguments\": [\"arg1\", \"arg2\"]}}\n```. If no tool is needed, respond with: ```json\n{{\"action\": \"no_tool\", \"response\": \"Your response here\"}}\n```"),
    MessagesPlaceholder("human_message")
])

# ... (Similarly define ChatPromptTemplates for delete_track, create_playlist, delete_playlist)

# System message (remains the same)
sys_msg = SystemMessage(content="You are a helpful assistant that helps users with Spotify operations.  You have access to tools to search for tracks.")

# Assistant Node (Updated to use prompts and dynamic assistant messages)
def assistant(state: SpotifyState):
    messages = state.get("messages", [])
    user_input = state.get("user_input")

    if state.get("current_node") == "get_playlists":
        prompt = playlist_selection_prompt.format(human_message=HumanMessage(content=user_input))
        messages.append(prompt)

    elif state.get("current_node") == "list_tracks":
        prompt = track_listing_prompt.format(human_message=HumanMessage(content=user_input))
        messages.append(prompt)

    elif state.get("current_node") == "add_track":
        prompt = add_track_prompt.format(human_message=HumanMessage(content=user_input))
        messages.append(prompt)

    # ... (Add other prompts for delete_track, create_playlist, delete_playlist)
    else:
      messages.append(HumanMessage(content=user_input))

    response = llm_with_tools.invoke(messages)
    messages.append(response)
    return {"messages": messages}, "process_assistant_response"

# Process Assistant Response Node (Updated to parse JSON)
def process_assistant_response(state: SpotifyState):
    assistant_response = state["messages"][-1].content  # Get the content of the last message
    try:
        json_response = json.loads(assistant_response)  # Parse JSON
        if json_response["action"] == "tool_call":
            tool_name = json_response["tool"]
            arguments = json_response["arguments"]

            if tool_name == "SearchTrack": # Check the tool name
              tool_result = search_track_tool(arguments[0], arguments[1]) # Call the tool with correct arguments
              if tool_result:
                return {"tool_result": tool_result}, "add_track"
              else:
                return {}, "track_not_found"

            # ... (Handle other tools similarly)

        elif json_response["action"] == "no_tool":
            # Handle no-tool case
            return {}, "list_tracks"  # Or appropriate next node

    except json.JSONDecodeError:
        print(f"Invalid JSON response: {assistant_response}")
        return {}, "invalid_action"  # Or handle the error as needed
    except KeyError as e:
        print(f"Missing key in JSON: {e}. Response: {json_response}")
        return {}, "invalid_action"
    except Exception as e: # Catch any other exceptions
        print(f"An unexpected error occurred: {e}")
        return {}, "invalid_action"
    return {}, "invalid_action"  # Default case

# ... (add_track and other nodes remain the same)

# ... (SpotifyState, build graph, interactive loop remain the same)

In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Build the graph
builder = StateGraph(SpotifyState)

builder.add_node("get_playlists", get_playlists) # Simplified node definition
builder.add_node("select_playlist", select_playlist)
builder.add_node("list_tracks", list_tracks)
builder.add_node("process_action", process_action)
builder.add_node("add_track", add_track)
builder.add_node("delete_track", delete_track)
builder.add_node("create_playlist", create_playlist)
builder.add_node("delete_playlist", delete_playlist)
builder.add_node("playlist_not_found", playlist_not_found)
builder.add_node("track_not_found", track_not_found)
builder.add_node("invalid_song_format", invalid_song_format)
builder.add_node("invalid_action", invalid_action)
#edges now...
builder.add_edge("get_playlists", "select_playlist")
builder.add_edge("select_playlist", "list_tracks")
builder.add_edge("select_playlist", "playlist_not_found")

builder.add_edge("list_tracks", "process_action")
builder.add_edge("process_action", "add_track")
builder.add_edge("process_action", "delete_track")
builder.add_edge("process_action", "create_playlist")
builder.add_edge("process_action", "delete_playlist")
builder.add_edge("process_action", "invalid_action")
builder.add_edge("process_action", END)  # User says "done"

builder.add_edge("add_track", "list_tracks")
builder.add_edge("add_track", "track_not_found")
builder.add_edge("add_track", "invalid_song_format")

builder.add_edge("delete_track", "list_tracks")
builder.add_edge("delete_track", "track_not_found")

builder.add_edge("create_playlist", "get_playlists")  # Refresh playlist list
builder.add_edge("delete_playlist", "get_playlists")  # Refresh playlist list

builder.add_edge("playlist_not_found", "get_playlists")
builder.add_edge("track_not_found", "list_tracks")
builder.add_edge("invalid_song_format", "add_track")
builder.add_edge("invalid_action", "list_tracks")


# Compile the graph
graph = builder.compile()





## resume from here... 

In [56]:
from langgraph.graph import StateGraph, END
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json

# ... (Spotify API initialization, LLM initialization, prompts, tool definitions, and SpotifyState class remain the same)

# Build the graph (Complete)
builder = StateGraph(SpotifyState)

# Define nodes
builder.add_node("assistant", assistant)
builder.add_node("process_assistant_response", process_assistant_response)
builder.add_node("add_track", add_track)
builder.add_node("delete_track", delete_track)  # Add other tool nodes
builder.add_node("create_playlist", create_playlist)
builder.add_node("delete_playlist", delete_playlist)
builder.add_node("get_playlists", get_playlists)
builder.add_node("select_playlist", select_playlist)
builder.add_node("list_tracks", list_tracks)
builder.add_node("track_not_found", track_not_found)
builder.add_node("invalid_song_format", invalid_song_format)
builder.add_node("invalid_action", invalid_action)
builder.add_node("playlist_not_found", playlist_not_found)

# Define edges (Complete)
builder.add_edge(START,"get_playlists")
builder.add_edge("get_playlists", "select_playlist")
builder.add_edge("select_playlist", "list_tracks")
builder.add_edge("select_playlist", "playlist_not_found")
builder.add_edge("list_tracks", "assistant")
builder.add_edge("assistant", "process_assistant_response")

builder.add_edge("process_assistant_response", "add_track")
builder.add_edge("process_assistant_response", "delete_track")  # Add edges for other tool nodes
builder.add_edge("process_assistant_response", "create_playlist")
builder.add_edge("process_assistant_response", "delete_playlist")
builder.add_edge("process_assistant_response", "list_tracks") # if no tool is called
builder.add_edge("process_assistant_response", "track_not_found")
builder.add_edge("process_assistant_response", "invalid_action")
builder.add_edge("add_track", "list_tracks")
builder.add_edge("delete_track", "list_tracks")  # Add return edges for other tool nodes
builder.add_edge("create_playlist", "get_playlists")
builder.add_edge("delete_playlist", "get_playlists")
builder.add_edge("track_not_found", "list_tracks")
builder.add_edge("invalid_song_format", "add_track")
builder.add_edge("invalid_action", "list_tracks")
builder.add_edge("playlist_not_found", "get_playlists")

# Compile the graph
graph = builder.compile()

# ... (Rest of the code, including the interactive loop, remains the same)

LET'S INVOKE!!

In [28]:

# Interactive loop (Corrected using graph.invoke())
state_filename = "spotify_state.json"
state = SpotifyState.load(state_filename)
state["current_node"] = state.get("current_node", "get_playlists")
state["messages"] = state.get("messages", [])

while True:
    print(f"Current Node: {state['current_node']}")  # For debugging

    if state["current_node"] == END:
        print("Goodbye!")
        state.save(state_filename)
        break

    if state["current_node"] in ["get_playlists", "select_playlist", "list_tracks"]:
        user_input = input("Enter your command: ")
        state["user_input"] = user_input

    # Invoke the graph
    try:
        state = graph.invoke(state)  # This is the crucial line!
    except Exception as e:
        print(f"Error during graph invocation: {e}")
        break  # Or handle the error as needed

    # Print any relevant output from the node (e.g., LLM response)
    if "messages" in state and state["messages"]:
        print(state["messages"][-1].content)

    state.save(state_filename)

Current Node: get_playlists
Error during graph invocation: Expected dict, got ({}, 'select_playlist')
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE


In [51]:
# ... (imports, Spotify API initialization, tool definitions, LLM initialization, prompts, SpotifyState remain the same)

# ... (graph building remains the same)



state_filename = "spotify_state.json"
state = SpotifyState.load(state_filename)
state["current_node"] = state.get("current_node", "get_playlists")
state["messages"] = state.get("messages", [])

while True:
    print(f"Current Node: {state['current_node']}")  # For debugging

    if state["current_node"] == END:
        print("Goodbye!")
        state.save(state_filename)
        break

    if state["current_node"] in ["get_playlists", "select_playlist", "list_tracks"]:
        user_input = input("Enter your command: ")
        state["user_input"] = user_input

    # Invoke the graph (Corrected - Crucial Change Here)
    try:
        if state["current_node"] == "get_playlists" and not state.get("playlists"):  # First invocation
            updated_state, next_node = get_playlists(state)  # Call get_playlists
            state.update(updated_state)  # Update the state with the returned changes
            state["current_node"] = next_node
        state = graph.invoke(state)  # NOW invoke the graph (after initialization)

    except Exception as e:
        print(f"Error during graph invocation: {e}")
        break


    # Print any relevant output from the node (e.g., LLM response)
    if "messages" in state and state["messages"]:
        print(state["messages"][-1].content)

    state.save(state_filename)

Current Node: get_playlists
Error during graph invocation: Expected dict, got ({'playlists': ['songs that feel like iris by the goo goo dolls (ID: 7Jm3IVBMpWYBGvM7LhAmt5)', 'Deep House ~2025 (ID: 7LVeXSb3I0HaEHyVB5xwMa)', 'doja cat best songs playlist (ID: 5YxpHUQM0f9bDFrpaSk84a)', 'RN (ID: 6AEZ3LLYSn6NifZTsmDy52)', 'Heavy Hardgroove Techno & Trance Set by ALBA FRANCH (ID: 0doBTaqvIhpQ6EZhKXd6SX)', 'Deep Melodic Electronic (ID: 5iKNZ3C7CGUzvaiZgx6Loz)', 'fashion killa type of vibe (ID: 3aC9U507hpy59ZJENoLdnb)', ' Progressive :: House :: Slow trance :: Techno  (ID: 1PCITdMOzifQXzPSSxMpzn)', 'alternative/indie rock jams (ID: 3UFTGPzA9kx3DPFkTf5Z4W)', '🔥erature (ID: 0E6zeXtfFpUEzSUpX9FAAr)', 'I HATE MODELS - Boiler Room 2024 (ID: 50pkqCal2DJhzTlSK7PVSa)', 'ALMOST HUMAN SESSIONS  (ID: 1and7h5SqpgAcUHNs6Z2NT)', 'Fast rap hard flows  (ID: 0zY5G11rddL26UPwtqoRag)', 'Best Rap Flows - Hip Hop (ID: 1dSQnCbtTAEtPdLo9OFw6T)', 'Seductive vibe;) (ID: 0MjH9fdQyO3AXEzgkKSrHW)', 'Melodic Techno 2025 🌙 

In [57]:
state_filename = "spotify_state.json"

try:
    with open(state_filename, "r") as f:
        loaded_state = json.load(f)
        state = SpotifyState.from_dict(loaded_state) # Load from the dict
except FileNotFoundError:
    state = SpotifyState()  # Create new if file doesn't exist

while True:
    print(f"Current Node: {state.current_node}")

    if state.current_node == END:
        print("Goodbye!")
        with open(state_filename, "w") as f:
            json.dump(state.to_dict(), f) # Save the dict
        break

    if state.current_node in ["get_playlists", "select_playlist", "list_tracks"]:
        user_input = input("Enter your command: ")
        state.user_input = user_input

    try:
      state_dict = state.to_dict() # convert to dict before invoking
      next_state_dict = graph.invoke(state_dict) # invoke the graph
      state = SpotifyState.from_dict(next_state_dict) # convert back to dataclass
    except Exception as e:
        print(f"Error during graph invocation: {e}")
        break

    if state.messages:
        print(state.messages[-1].content)  # Print last message

Current Node: get_playlists
Error during graph invocation: Expected dict, got ({'playlists': ['songs that feel like iris by the goo goo dolls (ID: 7Jm3IVBMpWYBGvM7LhAmt5)', 'Deep House ~2025 (ID: 7LVeXSb3I0HaEHyVB5xwMa)', 'doja cat best songs playlist (ID: 5YxpHUQM0f9bDFrpaSk84a)', 'RN (ID: 6AEZ3LLYSn6NifZTsmDy52)', 'Heavy Hardgroove Techno & Trance Set by ALBA FRANCH (ID: 0doBTaqvIhpQ6EZhKXd6SX)', 'Deep Melodic Electronic (ID: 5iKNZ3C7CGUzvaiZgx6Loz)', 'fashion killa type of vibe (ID: 3aC9U507hpy59ZJENoLdnb)', ' Progressive :: House :: Slow trance :: Techno  (ID: 1PCITdMOzifQXzPSSxMpzn)', 'alternative/indie rock jams (ID: 3UFTGPzA9kx3DPFkTf5Z4W)', '🔥erature (ID: 0E6zeXtfFpUEzSUpX9FAAr)', 'I HATE MODELS - Boiler Room 2024 (ID: 50pkqCal2DJhzTlSK7PVSa)', 'ALMOST HUMAN SESSIONS  (ID: 1and7h5SqpgAcUHNs6Z2NT)', 'Fast rap hard flows  (ID: 0zY5G11rddL26UPwtqoRag)', 'Best Rap Flows - Hip Hop (ID: 1dSQnCbtTAEtPdLo9OFw6T)', 'Seductive vibe;) (ID: 0MjH9fdQyO3AXEzgkKSrHW)', 'Melodic Techno 2025 🌙 